In [1]:
import hvplot.pandas
import numpy as np
import pandas as pd
import panel as pn

from ads_analytics import (
    get_facebook_ads_campaign_metrics,
    get_google_ads_campaign_metrics,
)
from google_analytics import get_landing_page_report
from hubspot_conversions import get_hubspot_conversions
from sales import get_sales_data

In [2]:
pn.extension(design="material")
pn.config.theme = "dark"

plot_opts = dict(responsive=True, min_height=400)

In [3]:
GLOBAL_FILTERS = {
    "date": "date",
    "campaign": "Campaign",
    "source": "Source",
    "medium": "Medium",
    "content": "Content",
    "term": "Term",
}

KEY_METRICS = {
    "ad_spend": "Ad spend",
    "ad_clicks": "Ad Clicks",
    "first_call_scheduled": "First Call scheduled",
    "verbal_agreement": "Verbal agreement (First call)",
    "placement_scheduled": "Placement scheduled",
    "sale": "Sale",
}

KEY_METRIC_FILTERS = {
    "ad_spend": "Ad spend",
    "ad_clicks": "Ad Clicks",
    "first_call_scheduled": "First Call scheduled",
    "verbal_agreement": "Verbal agreement (First call)",
    "placement_scheduled": "Placement scheduled",
    "sale": "Sale",
}

CONVERSION_ATTRIBUTION_CONVERSION_TYPES = {
    "ad_click": "Ad Click",
    "first_call": "First call",
    "verbal_agreement": "Verbal agreement (First call)",
    "placement_scheduled": "Placement scheduled",
    "sale": "Sale",
}

CONVERSION_ATTRIBUTION_BREAKDOWN = {
    "source": "Source",
    "medium": "Medium",
    "campaign": "Campaign",
    "content": "Content",
    "term": "Term",
}


In [4]:
def get_daily_data():
    hs = get_hubspot_conversions()
    sales = get_sales_data()
    fb = get_facebook_ads_campaign_metrics()
    ga = get_google_ads_campaign_metrics()
    lp = get_landing_page_report()
    data = pd.concat([hs, sales, fb, ga, lp])
    return data


In [5]:
data = get_daily_data()

[2025-04-14 09:34:42 - INFO] Retrieving deals from Hubspot
[2025-04-14 09:34:45 - INFO] Retrieved 850 deals from Hubspot in 2.87 seconds
[2025-04-14 09:34:45 - INFO] Retrieving first calls from Hubspot
[2025-04-14 09:38:33 - INFO] Retrieved 1397 first calls from Hubspot in 227.62 seconds
[2025-04-14 09:38:33 - INFO] Retrieving new calendly data since 2025-03-06 00:00:00
[2025-04-14 09:38:33 - INFO] Retrieving calendly data
[2025-04-14 09:39:53 - INFO] Retrieved 254 meetings from calendly in 80.07 seconds
[2025-04-14 09:39:53 - INFO] Retrieving sales data from Google Sheet
[2025-04-14 09:39:55 - INFO] Retrieved sales data in 2.08 seconds
[2025-04-14 09:39:55 - INFO] Retrieving new calendly data since 2025-03-06 00:00:00
[2025-04-14 09:39:55 - INFO] Retrieving calendly data
[2025-04-14 09:39:55 - INFO] Retrieved 254 calendly data from Hubspot in 0.03 seconds
[2025-04-14 09:39:55 - INFO] Retrieving Facebook Ads campaign metrics
[2025-04-14 09:40:22 - INFO] Retrieved Facebook Ads campaign 

In [6]:
# Ensure date column is datetime
data["date"] = pd.to_datetime(data["date"])

# Create widgets for filters and comparisons
date_range = pn.widgets.DateRangeSlider(
    name="Date Range",
    start=data["date"].min().date(),
    end=data["date"].max().date(),
    value=(data["date"].min().date(), data["date"].max().date()),
)

conversion_filter = pn.widgets.MultiChoice(
    name="Conversion Types",
    options=[
        "Ad Click",
        "First call",
        "Verbal agreement (First call)",
        "Placement scheduled",
        "Sale",
    ],
    value=[
        "ad_click",
        "first_call",
        "verbal_agreement",
        "placement_scheduled",
        "sale",
    ],
)

time_agg = pn.widgets.RadioButtonGroup(
    name="Time Aggregation", options=["daily", "weekly", "monthly"], value="daily"
)

# UTM Parameter filters
utm_campaign_filter = pn.widgets.MultiChoice(
    name="Campaign",
    options=list(data["Campaign"].dropna().unique()),
)

utm_source_filter = pn.widgets.MultiChoice(
    name="Source",
    options=list(data["Source"].dropna().unique()),
)

utm_medium_filter = pn.widgets.MultiChoice(
    name="Medium",
    options=list(data["Medium"].dropna().unique()),
)

# Comparison selector
comparison_selector = pn.widgets.Select(
    name="Compare By",
    options=[
        "None",
        "Campaign",
        "Source",
        "Medium",
        "Content",
        "Term",
    ],
    value="None",
)

KeyError: 'date'

In [11]:
# Function to create the plot
def create_key_metrics_plot(aggregated_analytics_raw, compare_by="None"):
    return pn.pane.Markdown("No data to display.")


# Update function for the dashboard
@pn.depends(
    date_range.param.value,
    conversion_filter.param.value,
    time_agg.param.value,
    utm_campaign_filter.param.value,
    utm_source_filter.param.value,
    utm_medium_filter.param.value,
    comparison_selector.param.value,
)
def update_key_metrics_chart(
    date_range,
    conversion_types,
    time_agg,
    utm_campaign,
    utm_source,
    utm_medium,
    compare_by,
):
    if not conversion_types:  # Ensure at least one conversion type is selected
        return pn.pane.Markdown("Please select at least one conversion type.")

    data = pd.DataFrame()

    plot = create_key_metrics_plot(data, compare_by)
    return plot

In [12]:
# Function to create the plot
def create_conversion_attribution_plot(aggregated_analytics_raw, compare_by="None"):
    # TODO
    return pn.pane.Markdown("No data to display.")


# Update function for the dashboard
@pn.depends(
    date_range.param.value,
    conversion_filter.param.value,
    time_agg.param.value,
    utm_campaign_filter.param.value,
    utm_source_filter.param.value,
    utm_medium_filter.param.value,
    comparison_selector.param.value,
)
def update_conversion_attribution_chart(
    date_range,
    conversion_types,
    time_agg,
    utm_campaign,
    utm_source,
    utm_medium,
    compare_by,
):
    if not conversion_types:  # Ensure at least one conversion type is selected
        return pn.pane.Markdown("Please select at least one conversion type.")

    data = pd.DataFrame()
    # TODO

    plot = create_conversion_attribution_plot(data, compare_by)
    return plot

In [13]:
# Create the dashboard layout

key_metrics_chart_filters = pn.WidgetBox(
    "## Filters",
    conversion_filter,
    time_agg,
    pn.layout.Divider(),
    "### UTM Parameters",
    utm_campaign_filter,
    utm_source_filter,
    utm_medium_filter,
    pn.layout.Divider(),
    "### Comparison",
    comparison_selector,
    width=330,
    styles={"padding": "10px"},
)
key_metrics_chart = pn.Column(
    "## Key Metrics Evolution and Trends",
    update_key_metrics_chart,
    sizing_mode="stretch_width",
)
key_metrics_panel = pn.Row(
    key_metrics_chart_filters, key_metrics_chart, sizing_mode="stretch_width"
)

In [14]:
conversion_attribution_chart_filters = pn.WidgetBox(
    "## Filters",
    conversion_filter,
    time_agg,
    pn.layout.Divider(),
    "### UTM Parameters",
    utm_campaign_filter,
    utm_source_filter,
    utm_medium_filter,
    pn.layout.Divider(),
    "### Comparison",
    comparison_selector,
    width=330,
    styles={"padding": "10px"},
)
conversion_attribution_chart = pn.Column(
    "## Conversion Attribution",
    update_conversion_attribution_chart,
    sizing_mode="stretch_width",
)
conversion_attribution_panel = pn.Row(
    conversion_attribution_chart_filters,
    conversion_attribution_chart,
    sizing_mode="stretch_width",
)


In [15]:
# Instantiate the template with widgets displayed in the sidebar
template = pn.template.EditableTemplate(
    editable=True,
    title="EditableTemplate",
    sidebar=[date_range],
)

template.main.extend([key_metrics_panel, conversion_attribution_panel])
template.servable()

EditableTemplate
    [editor] TemplateEditor(design=<class 'panel.theme.materi...)
    [js_area] HTML(None, design=<class 'panel.theme.materi..., height=0, margin=0, sizing_mode='fixed', width=0)
    [actions] TemplateActions()
    [browser_info] BrowserInfo()
    [busy_indicator] LoadingSpinner(height=20, width=20)
    [nav-133570465594832] DateRangeSlider(design=<class 'panel.theme.materi..., end=datetime.date(2025, 3, 6), name='Date Range', start=datetime.date(2024, ..., value=(datetime.date(2024, ..., value_end=datetime.date(2025, 3, 6), value_start=datetime.date(2024, ...)
    [main-133570463067328] Row(design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
        [0] WidgetBox(design=<class 'panel.theme.materi..., styles={'padding': '10px'}, width=330)
            [0] Markdown(str, design=<class 'panel.theme.materi...)
            [1] MultiChoice(design=<class 'panel.theme.materi..., name='Conversion Types', options=['Ad Click', 'First call',...])
            [2] RadioButtonGroup(design=<class 'panel.theme.materi..., name='Time Aggregation', options=['daily', 'weekly', ...], value='daily')
            [3] Divider(design=<class 'panel.theme.materi...)
            [4] Markdown(str, design=<class 'panel.theme.materi...)
            [5] MultiChoice(design=<class 'panel.theme.materi..., name='Campaign', options=['HM - TOF - ANGLE TESTING...])
            [6] MultiChoice(design=<class 'panel.theme.materi..., name='Source', options=['fb', 'ig', 'facebook', ...])
            [7] MultiChoice(design=<class 'panel.theme.materi..., name='Medium', options=['Facebook_Mobile_Feed', ...])
            [8] Divider(design=<class 'panel.theme.materi...)
            [9] Markdown(str, design=<class 'panel.theme.materi...)
            [10] Select(design=<class 'panel.theme.materi..., name='Compare By', options=['None', 'Campaign', ...], value='None')
        [1] Column(design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
            [0] Markdown(str, design=<class 'panel.theme.materi...)
            [1] ParamFunction(function, _pane=Markdown, defer_load=False, design=<class 'panel.theme.materi...)